In [11]:
# Импорт библиотек
import pandas as pd
import seaborn as sns
from sklearn.ensemble import RandomForestRegressor

In [12]:
# Чтение файлов

df_train = pd.read_excel('class_A.xlsx')
df_test = pd.read_excel('class_B.xlsx')
df_predict = pd.read_excel('class_C.xlsx')

# Создание отдельных тренировочных, тестовых выборок и выборки для прогнозирования успеваемости по каждому из трех предметов

df_train_db, df_test_db, df_predict_db = df_train.iloc[:,6:15], df_test.iloc[:,6:15], df_predict.iloc[:,6:15]
df_train_matstat, df_test_matstat, df_predict_matstat = df_train.iloc[:,16:23], df_test.iloc[:,16:23], df_predict.iloc[:,16:23]
df_train_quant, df_test_quant, df_predict_quant = df_train.iloc[:,24:36], df_test.iloc[:,24:36], df_predict.iloc[:,24:36]

df_predict_db.head()

,ЛР1(10),ЛР2(15),ЛР3(10),ДЗ1(20),ДЗ2(10),ДЗ3(15),Зачетная работа(20),ИТОГО,ОЦЕНКА
0,4,7,5,15,7,15,NaN,NaN,NaN
1,7,0,7,13,6,11,NaN,NaN,NaN
2,7,9,2,17,10,15,NaN,NaN,NaN
3,5,15,9,15,4,7,NaN,NaN,NaN
4,2,15,2,19,4,15,NaN,NaN,NaN


In [13]:
# Меняем тип столбцов для устранения возникновения ошибок в дольнейшей работе кода
df_predict[['Зачетная работа(20)', 'ИТОГО', 'ОЦЕНКА', 'Экзамен(40)', 'ИТОГО.1', 'ОЦЕНКА.1', 'Экзамен(40).1', 'ИТОГО.2', 'ОЦЕНКА.2']]=df_predict[['Зачетная работа(20)', 'ИТОГО', 'ОЦЕНКА', 'Экзамен(40)', 'ИТОГО.1', 'ОЦЕНКА.1', 'Экзамен(40).1', 'ИТОГО.2', 'ОЦЕНКА.2']].astype('int', errors='ignore')
df_predict.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20 entries, 0 to 19
Data columns (total 36 columns):
 #   Column                                           Non-Null Count  Dtype         
---  ------                                           --------------  -----         
 0   №                                                20 non-null     int64         
 1   Фамилия                                          20 non-null     object        
 2   Имя                                              20 non-null     object        
 3   Отчество                                         20 non-null     object        
 4   Дата рождения                                    20 non-null     datetime64[ns]
 5   Базы данных                                      0 non-null      float64       
 6   ЛР1(10)                                          20 non-null     int64         
 7   ЛР2(15)                                          20 non-null     int64         
 8   ЛР3(10)                                   

In [14]:
# Обучаем модели по отдельным предметам на тренировочной выборке и проверяем их на тестовой

In [15]:
# Базы данных

# Обучаем
model_db = RandomForestRegressor()
model_db.fit(df_train_db.iloc[:,0:6].values.reshape(-1,6), y=df_train_db['Зачетная работа(20)'].values)

# Добавляем столбец Predict для сверения со столбцом 'Зачетная работа'
df_test_db['Predict']=model_db.predict(df_test_db.iloc[:,0:6].values.reshape(-1,6))

df_test_db.head(15)

,ЛР1(10),ЛР2(15),ЛР3(10),ДЗ1(20),ДЗ2(10),ДЗ3(15),Зачетная работа(20),ИТОГО,ОЦЕНКА,Predict
0,9,1,4,15,5,15,11,60,3,11.33
1,5,13,10,15,8,12,15,78,4,12.77
2,5,10,7,14,7,7,6,56,3,12.31
3,7,6,9,19,4,6,11,62,3,10.17
4,2,11,7,18,10,6,12,66,3,11.20
5,7,2,1,18,3,10,9,50,3,8.17
6,4,8,3,15,3,6,8,47,2,8.22
7,6,6,10,12,9,7,11,61,3,12.98
8,10,11,9,15,5,13,15,78,4,12.55
9,3,11,1,16,4,14,10,59,3,7.99


In [16]:
# Предсказываем возможные результаты сессии у группы С

df_predict_db['Зачетная работа(20)']=model_db.predict(df_predict_db.iloc[:,0:6].values.reshape(-1,6))
df_predict_db['ИТОГО']=df_predict_db.iloc[:,0:7].sum(axis=1)

# Составляем соответствие оценки рейтингу
df_predict_db['ОЦЕНКА']=[(5 if score_db >=85 else 4) if score_db >=70 else (3 if score_db >=50 else 2) for score_db in df_predict_db['ИТОГО']]

df_predict_db.head(15)

,ЛР1(10),ЛР2(15),ЛР3(10),ДЗ1(20),ДЗ2(10),ДЗ3(15),Зачетная работа(20),ИТОГО,ОЦЕНКА
0,4,7,5,15,7,15,10.45,63.45,3
1,7,0,7,13,6,11,12.89,56.89,3
2,7,9,2,17,10,15,9.46,69.46,3
3,5,15,9,15,4,7,11.38,66.38,3
4,2,15,2,19,4,15,8.13,65.13,3
5,4,4,8,17,7,12,11.37,63.37,3
6,8,14,3,16,7,7,11.61,66.61,3
7,5,1,2,18,8,8,12.06,54.06,3
8,4,3,7,13,9,13,13.44,62.44,3
9,3,8,10,13,7,15,11.01,67.01,3


In [17]:
# Теория вероятностей и математическая статистика

# Обучаем
model_matstat = RandomForestRegressor()
model_matstat.fit(df_train_matstat.iloc[:,0:4].values.reshape(-1,4), y=df_train_matstat['Экзамен(40)'].values)

# Добавляем столбец Predict для сверения со столбцом 'Экзамен'
df_test_matstat['Predict']=model_matstat.predict(df_test_matstat.iloc[:,0:4].values.reshape(-1,4))

df_test_matstat.head(15)

,КР1(15),КР2(20),КР3(15),КР4(10),Экзамен(40),ИТОГО.1,ОЦЕНКА.1,Predict
0,9,15,8,8,37,77,4,21.75
1,12,9,12,10,27,70,4,25.12
2,6,13,7,10,17,53,2,15.82
3,6,13,8,5,23,55,3,16.36
4,3,14,13,5,19,54,2,20.36
5,4,8,14,8,21,55,3,19.10
6,6,5,11,10,20,52,2,18.63
7,8,8,11,10,22,59,3,22.54
8,3,14,14,10,37,78,4,21.12
9,0,8,14,5,16,43,2,19.64


In [18]:
# Предсказываем возможные результаты сессии у группы С

df_predict_matstat['Экзамен(40)']=model_matstat.predict(df_predict_matstat.iloc[:,0:4].values.reshape(-1,4))
df_predict_matstat['ИТОГО.1']=df_predict_matstat.iloc[:,0:5].sum(axis=1)

# Составляем соответствие оценки рейтингу
df_predict_matstat['ОЦЕНКА.1']=[(5 if score_matstat >=85 else 4) if score_matstat >=70 else (3 if score_matstat >=50 else 2) for score_matstat in df_predict_matstat['ИТОГО.1']]
df_predict_matstat.loc[df_predict_matstat['Экзамен(40)']<20, 'ОЦЕНКА.1'] = 2

df_predict_matstat.head(15)

,КР1(15),КР2(20),КР3(15),КР4(10),Экзамен(40),ИТОГО.1,ОЦЕНКА.1
0,15,14,8,6,23.01,66.01,3
1,0,19,15,7,22.35,63.35,3
2,3,16,15,6,19.47,59.47,2
3,10,8,8,8,20.41,54.41,3
4,8,7,10,10,22.33,57.33,3
5,0,20,15,9,23.08,67.08,3
6,4,12,12,6,20.00,54.00,3
7,4,11,12,8,19.24,54.24,2
8,4,14,5,5,18.33,46.33,2
9,12,20,8,7,22.27,69.27,3


In [19]:
# Квантовая физика

# Обучаем
model_quant = RandomForestRegressor()
model_quant.fit(df_train_quant.iloc[:,0:9].values.reshape(-1,9), y=df_train_quant['Экзамен(40).1'].values)

# Добавляем столбец Predict для сверения со столбцом 'Экзамен'
df_test_quant['Predict']=model_quant.predict(df_test_quant.iloc[:,0:9].values.reshape(-1,9))

df_test_quant.head(15)

,ЛР1(4),ЛР2(4),ЛР3(4),ЛР4(4),ЛР(5),ТЕСТ1(5),ТЕСТ2(5),КР1(15).1,КР2(15),Экзамен(40).1,ИТОГО.2,ОЦЕНКА.2,Predict
0,4,0,0,1,3,3,2,6,10,24,53,3,27.45
1,2,1,0,3,1,5,0,6,9,23,50,3,25.42
2,1,1,1,4,4,3,0,9,10,26,59,3,25.15
3,1,4,4,4,1,0,1,13,8,28,64,3,23.94
4,1,2,4,1,1,1,0,10,14,27,61,3,29.04
5,4,4,2,0,3,3,0,11,6,26,59,3,19.32
6,0,2,2,4,4,5,2,6,9,27,61,3,24.41
7,0,3,0,2,0,5,1,6,8,22,47,2,25.61
8,0,1,1,1,4,3,4,7,15,28,64,3,31.21
9,0,1,4,0,1,0,0,10,11,23,50,3,26.74


In [20]:
# Предсказываем возможные результаты сессии у группы С

df_predict_quant['Экзамен(40).1']=model_quant.predict(df_predict_quant.iloc[:,0:9].values.reshape(-1,9))
df_predict_quant['ИТОГО.2']=df_predict_quant.iloc[:,0:10].sum(axis=1)

# Составляем соответствие оценки рейтингу
df_predict_quant['ОЦЕНКА.2']=[(5 if score_quant >=85 else 4) if score_quant >=70 else (3 if score_quant >=50 else 2) for score_quant in df_predict_quant['ИТОГО.2']]
df_predict_quant.loc[df_predict_quant['Экзамен(40).1']<20, 'ОЦЕНКА.2'] = 2

df_predict_quant.head(15)

,ЛР1(4),ЛР2(4),ЛР3(4),ЛР4(4),ЛР(5),ТЕСТ1(5),ТЕСТ2(5),КР1(15).1,КР2(15),Экзамен(40).1,ИТОГО.2,ОЦЕНКА.2
0,4,3,1,2,1,4,5,5,11,29.83,65.83,3
1,2,1,4,3,4,2,4,14,5,20.25,59.25,3
2,3,1,0,3,2,5,2,14,10,29.86,69.86,3
3,2,3,1,1,0,4,2,6,15,30.47,64.47,3
4,1,1,0,2,2,0,1,11,13,28.21,59.21,3
5,1,1,0,4,4,5,5,15,8,28.98,71.98,4
6,3,4,0,4,4,2,4,15,6,20.55,62.55,3
7,4,3,0,3,2,3,4,14,14,38.26,85.26,5
8,1,3,2,2,1,4,0,14,8,29.37,64.37,3
9,3,2,1,1,0,1,2,11,8,27.95,56.95,3
